In [1]:
import pandas as pd

In [2]:
group_info_AFT2000 = pd.read_csv('group_info_AFT2000.csv')

In [3]:
group_info_AFT2000.sample(5)

,MenteeID,StopYear,GroupSize,MentorID,mag,author_id,first_pub_year,last_pub_year,career_len
74960,18584,2008,2,18588,2.980480e+09,NaN,NaN,NaN,NaN
50079,94889,2010,4,10815,1.186787e+09,A1186786555,2000.0,2021.0,21.0
80748,4638,2003,1,19833,2.473610e+09,A2473610168,2005.0,2022.0,17.0
63404,214790,2003,11,212622,2.499547e+09,A2499546652,2001.0,2015.0,14.0
36943,50707,2004,15,7103,2.488096e+09,A2488096353,2004.0,2004.0,0.0


In [4]:
group_info = pd.read_csv('group_info.csv')

In [5]:
group_info.head(5)

,work_id,author_position,author_id,institution_id,publication_year,first_pub_year,scientific_age,group_size,mentor_id,current_year,ratio_institution,last_pub_year,career_len,mentor_first_pub_year,mentor_scientific_age
0,W2031577749,middle,A2491431510,I1294671590,1972,1970,2,2,A367921,1972,1.0,1976.0,6.0,1971,1
1,W2081199337,middle,A2491431510,I1294671590,1972,1970,2,2,A367921,1972,1.0,1976.0,6.0,1971,1
2,W2081199337,middle,A2724725885,I1294671590,1972,1970,2,2,A367921,1972,1.0,1976.0,6.0,1971,1
3,W2031577749,middle,A2491431510,I1294671590,1972,1970,2,2,A367921,1973,1.0,1976.0,6.0,1971,2
4,W2081199337,middle,A2491431510,I1294671590,1972,1970,2,2,A367921,1973,1.0,1976.0,6.0,1971,2


##### Find OpenAlex ID of MentorID

In [6]:
mentor_AFT = list(set(group_info_AFT2000.MentorID))

In [7]:
researcher = pd.read_csv('researcher.csv')

In [8]:
researcher.head(5)

,PID,FirstName,MiddleName,LastName,Institution,InstitutionMAGID,ResearchArea,ORCID,MAGAuthorID
0,1,STEPHEN,V,DAVID,Oregon Health and Science University,165690674.0,"neuro,csd,bme",0000-0003-4135-3104,2.171828e+09
1,2,BENJAMIN,Y,HAYDEN,"University of Minnesota, Twin Cities",130238516.0,neuro,0000-0002-7678-4281,2.082173e+09
2,3,JACK,L,GALLANT,"University of California, Berkeley",95457486.0,"neuro,psych",NaN,2.110166e+09
3,4,BENJAMIN,NaN,WILLMORE,University of Oxford,40120149.0,neuro,NaN,2.183264e+09
4,5,KENDRICK,NORRIS,KAY,"University of Minnesota, Twin Cities",130238516.0,neuro,NaN,2.156587e+09


In [9]:
mentor_MAG = researcher[['PID','MAGAuthorID']][(researcher.PID.isin(mentor_AFT))
                                              &(~researcher.MAGAuthorID.isnull())]

In [10]:
len(mentor_MAG)

15992

In [11]:
mentor_MAG.head(3)

,PID,MAGAuthorID
2,3,2.110166e+09
6,7,2.052279e+09
12,13,2.183534e+09


In [18]:
mentor_MAG = mentor_MAG.rename(columns={'MAGAuthorID':'mag'})

In [19]:
mentors = list(mentor_MAG.mag)

In [13]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}?charset=utf8mb4"
                       .format(user="openalex_ym",
                               pw="openalex.ym",
                               host="172.18.18.167",
                               port=3306,
                               db="openalex2022"),
                       )

In [14]:
magId = [int(k) for k in mentors]

In [15]:
magId_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = magId[start_point:end_point]
        magId_groups.append(temp)
    except:
        temp = magId[start_point:]
        magId_groups.append(temp)
        
    start_point = end_point
    if end_point>len(magId):
        break

In [16]:
l = len(magId_groups)
for i in range(l):
    temp =  magId_groups[i]
    sql_query = """select authors_ids.author_id, authors_ids.mag 

                    from authors_ids 

                    where authors_ids.mag in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))

data = tp

In [17]:
data.head(5)

,author_id,mag
0,A461721,461721
1,A1121383,1121383
2,A1750657,1750657
3,A2626456,2626456
4,A3881444,3881444


In [20]:
mentor_MAG = mentor_MAG.merge(data,on='mag',how='left')

In [23]:
mentor_MAG.head(3)

,MentorID,mentor_mag,mentor_id
0,3,2.110166e+09,A2110165986
1,7,2.052279e+09,A2052278781
2,13,2.183534e+09,A2183534167


In [22]:
mentor_MAG.columns = ['MentorID','mentor_mag','mentor_id']

In [24]:
group_info_AFT2000 = group_info_AFT2000.merge(mentor_MAG,on='MentorID',how='left')

In [25]:
group_info_AFT2000.sample(5)

,MenteeID,StopYear,GroupSize,MentorID,mag,author_id,first_pub_year,last_pub_year,career_len,mentor_mag,mentor_id
7215,4017,2003,3,839,1.867895e+09,A1867894562,1999.0,2022.0,23.0,2.428469e+09,A2428468760
87961,451514,2010,8,448215,NaN,NaN,NaN,NaN,NaN,2.102067e+09,A2102066616
55196,637280,2007,2,635312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48471,505558,2000,7,501670,2.131129e+09,A2131128762,1995.0,1996.0,1.0,2.222329e+09,A2222329283
54881,78133,2008,1,78134,6.623292e+08,A662329191,2008.0,2022.0,14.0,2.649951e+09,A2649950932


In [26]:
group_info_AFT2000.to_csv('group_info_AFT2000.csv',index=0)